In [9]:
import automatization
from automatization.ple_statistics import PleAuto
import pandas as pd
import os
import json

In [ ]:
# check scanning range scaling
high_finesse_wavemeter_remote.start_acquisition()
high_finesse_wavemeter_remote._wavelength_buffer = []
wave_buff = high_finesse_wavemeter_remote.get_wavelength_buffer()
factor_sigma = float((np.max(np.array(wave_buff)) - np.min(np.array(wave_buff)) )/ 0.02)
factor_sigma

In [ ]:
factor_sigma = 1.04

In [1]:
pa = PleAuto(
    ple_gui, 
    ple_optimize_logic, 
    poi_manager_logic, 
    scanning_optimize_logic, 
    spectrometer, 
    spectrometerlogic,
    cobolt,
    ibeam_smart
)

1

In [ ]:
powers = [300, 270, 240, 220 , 200, 170, 150, 140, 130, 120, 110, 100, 80]
folder = r"C:\Users\yy3\Documents\data\SnV\cooldown_06_03_2023_95NA\197\2R\poiA"
os.mkdir(folder:=os.path.join(folder, r"automated_measurements"))
results = []
for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    results_poi = {}
    os.mkdir(folder_defect := os.path.join(folder, poi_name))
    
    # go to a defect
    results_poi["poi_name"] = poi_name
    results_poi["xyz"] = tuple(float(coord) for coord in poi_manager_logic.poi_anchors[poi_name])
    ibeam_smart.enable()
    pa.go_to_poi(poi_name)

    #set max resonant power
    pa.set_resonant_power(power = 300)
    # turn on the CW repump
    ibeam_smart.disable()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 10)
    #perform one ple scan to determine if there is a PLE
    res = pa.do_ple_scan(lines = 1, in_range = None)
    if res["center"].stderr > 1e3 or res["sigma"].stderr > res["sigma"].value: 
        continue
    #since there is a PLE -- optimize_all
    pa.go_to_ple_target(res["center"].value)
    #get the center wavelength:
    wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
    results_poi["wavelength, THz"] = float(wavelength)
    time.sleep(0.5)

    pa.optimize_all()

    #do a san again to determine a finer range to scan
    res = pa.do_ple_scan(lines = 5)
    #it makes sense to save here
    pa.save_ple(tag = "full_range",
             poi_name=poi_name, folder_name=folder_defect)

    fine_scan_range = ( 
        res["center"].value - 5 * res["sigma"].value ,
        res["center"].value + 5 * res["sigma"].value
                    )
    pa.set_resonant_power(power = 270)

    res = pa.do_ple_scan(lines = 5, in_range=fine_scan_range)

    pa.save_ple(tag = "5sigma_range",
             poi_name=poi_name, folder_name=folder_defect)

    fine_scan_range = ( 
        res["center"].value - 4 * res["sigma"].value ,
        res["center"].value + 4 * res["sigma"].value
                    )
    
    res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
    # here we have a somewhat fine range for the scan. 
    #Let's perform a saturation scan with a blue cw repump ON
    os.mkdir(saturation_folder := os.path.join(folder_defect, "saturation"))

    idx_no_ple = None

    for idx, power in enumerate(powers):
        pa.go_to_ple_target(res["center"].value)
        pa.set_resonant_power(power = power)
        pa.optimize_ple()
        res = pa.do_ple_scan(lines = 10, in_range=fine_scan_range)
        #save_results
        pa.save_ple(tag = f"{power}",
            poi_name=poi_name, folder_name=saturation_folder)
        #save_plots
        
        if res["center"].stderr > 1e3 or res["sigma"].stderr * 3 > res["sigma"].value: 
            #ple is gone.
            idx_no_ple = idx
            break
        time.sleep(1)
        results_poi.update({"saturation": 
                            {f"{power}":
                             {"scan_data": ple_data_logic.last_saved_files_paths,
                              "sigma": str(res["sigma"].value) + " ± " str(res["sigma"].stderr)
                              }
                             }})

    if idx_no_ple is None:
        continue
    # photostability study

    pa.set_resonant_power(power = powers[idx_no_ple - 2]) #take the last visible ple power
    #reionize with blue
    os.mkdir(photostability := os.path.join(folder_defect, "photostability_405"))

  
    for i in range(3):
        #walk off resonance:
        pa.go_to_ple_target(0)
        time.sleep(0.5)
        cobolt.enable_modulated()
        cobolt.set_laser_modulated_power(power = 20)
        time.sleep(2)
        cobolt.disable_modulated()
        ibeam_smart.disable()
        res = pa.do_ple_scan(lines = 50, in_range=fine_scan_range)
        #cobolt.set_laser_modulated_power(power = 10)
        pa.save_ple(tag = "photostability", folder_name=photostability) 

    #reionize with green
    os.mkdir(photostability := os.path.join(folder_defect, "photostability_520"))
    for i in range(3):
        #walk off resonance:
        pa.go_to_ple_target(0)
        time.sleep(0.5)
        ibeam_smart.enable()
        time.sleep(2)
        cobolt.disable_modulated()
        ibeam_smart.disable()
        
        res = pa.do_ple_scan(lines = 20, in_range=fine_scan_range)
        #cobolt.set_laser_modulated_power(power = 10)
        save_ple(tag = "photostability", folder_name=photostability) 
    
    with open(os.path.join(folder_defect, f'results_{poi_name}.txt'), 'w') as file:
        file.write(json.dumps(results_poi)) # use `json.loads` to do the reverse
    #with open(os.path.join(folder_defect, f'results_{poi_name}.txt')) as file:
    #results = json.load(file) # use `json.loads` to do the reverse

all_results = pd.DataFrame(results)
all_results.to_csv("ple_statistics" , sep=',')